Загружаем нужные библиотеки:

In [2]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point
import folium
import os
import pandas as pd
from IPython.display import display, HTML
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from functools import partial

##Шаг 1. Подготовка данных
Разархивировать и просмотреть все исходные данные.

Загрузить геоданные (GeoJSON)

Убедиться, что все данные имеют правильные координаты и геометрии.

##Загрузка слоёв:

*чтобы путь был универсальным(глобальным) создан репозиторий, а в нем папка (data) с файлами с данными

Создаем словарь, присваивая ссылке на файл(пути) имя переменной

In [3]:
geojson_urls = {
    "region_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/arhangelskaya_oblast.geojson",
    "people_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/base_obl_people.geojson",
    "people_3000_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/base_obl_people_3000.geojson",
    "buildings_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/buildings.geojson",
    "roads_bad_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/mun_obr_all_bad.geojson",
    "arctic_municipalities_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/mun_obr_arctic.geojson",
    "education_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/objects_education.geojson",
    "tourism_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/objects_tourism.geojson",
    "zdrav_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/objects_zdrav.geojson",
    "slow_roads_gdf": "https://raw.githubusercontent.com/vetochka12345/civilization-8/main/data/slow_roads_lines.geojson"
}

gdfs = {}

for name, url in geojson_urls.items():
  gdfs[name] = gpd.read_file(url)

c:\Users\smirn\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 1753 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


чтобы было удобнее заменим gdfs[name] на более короткую переменную

In [4]:
region=gdfs['region_gdf']
people=gdfs["people_gdf"]
people_3000=gdfs['people_3000_gdf']
buildings=gdfs['buildings_gdf']
roads_bad=gdfs['roads_bad_gdf']
arctic_municipalities=gdfs['arctic_municipalities_gdf']
education=gdfs['education_gdf']
tourism=gdfs['tourism_gdf']
zdrav=gdfs['zdrav_gdf']
slow_roads=gdfs['slow_roads_gdf']


Соединим people в один df при условии одинаковой сис-мы к-т

In [5]:
people["people"] = 0  #т.к. в файле people нет столбца с числ-тью населения, а в people_3000 есть, добавим этот столбец со значением 0(поскольку нам ничего не известно)
if people.crs != people_3000.crs:
    people_3000 = people_3000.to_crs(people.crs)

population = pd.concat([people, people_3000], ignore_index=True)

Список слоёв:

In [6]:
geo_vars = ['region', 'population', 'buildings', 'roads_bad',
            'arctic_municipalities', 'education', 'tourism', 'zdrav', 'slow_roads']

In [7]:
region_shape = region.unary_union  #объединяем в одну геометрию

print("Удаление объектов вне границ региона:\n")

for name in geo_vars:
    if name not in globals():
        print(f"{name}: переменная не найдена")
        continue

    inshape = globals()[name]

    if "geometry" not in inshape.columns:
        print(f"{name}: отсутствует геометрия")
        continue

    before = len(inshape)
    #оставляем объекты, которые находятся внутри или пересекаются с регионом
    inshape = inshape[inshape.geometry.intersects(region_shape)]
    after = len(inshape)

    globals()[name] = inshape

    print(f"{name}: удалено {before - after}")


C:\Users\smirn\AppData\Local\Temp\ipykernel_5336\1413864090.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  region_shape = region.unary_union  #объединяем в одну геометрию


Удаление объектов вне границ региона:

region: удалено 0
population: удалено 786
buildings: удалено 0
roads_bad: удалено 61
arctic_municipalities: удалено 64
education: удалено 0
tourism: удалено 0
zdrav: удалено 0
slow_roads: удалено 0


##Проверка данных

**Проверка crs:**

In [8]:
incorrect_crs_report = []

base_crs=region.crs

for var_name in geo_vars:
    gdf = locals()[var_name]
    if gdf.crs != base_crs:
        incorrect_crs_report.append((var_name, gdf.crs))  # сохраняем имя и старую CRS
        locals()[var_name] = gdf.to_crs(base_crs) # перепроецируем

if incorrect_crs_report:
    print("Перепроецированы в CRS", base_crs)
    for name, old_crs in incorrect_crs_report:
        print(f"- {name}: было {old_crs}")
else:
    print("Везде нужная CRS:", base_crs)

Везде нужная CRS: EPSG:3857


**Проверка геометрии**

Удаление пустых геометрий и исправление невалидных

In [9]:
def check(geo_list):

    print("Проверка геометрий в слоях:\n")

    for name in geo_list:
        if name not in globals():
            print(f"{name}: переменная не найдена\n")
            continue

        df = globals()[name]
        if "geometry" not in df.columns:
            print(f"{name}: нет колонки geometry\n")
            continue

        empty_count = df["geometry"].isna().sum()
        df = df.dropna(subset=["geometry"])

        invalid = ~df.is_valid
        fixed = invalid.sum()
        if fixed > 0:
            df.loc[invalid, "geometry"] = df.loc[invalid, "geometry"].buffer(0)

        globals()[name] = df

        print(f"{name}:")
        print(f"  удалено пустых геометрий: {empty_count}")
        print(f"  исправлено невалидных: {fixed}\n")



In [10]:
check(geo_vars)

Проверка геометрий в слоях:

region:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

population:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

buildings:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

roads_bad:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

arctic_municipalities:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

education:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

tourism:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

zdrav:
  удалено пустых геометрий: 0
  исправлено невалидных: 0

slow_roads:
  удалено пустых геометрий: 0
  исправлено невалидных: 0



**Проверка и исправление атрибутивных данных**


In [11]:
attribute_cleanup_report = []

for var_name in geo_vars:
    gdf = locals()[var_name]

    original_cols = gdf.columns.tolist()
    duplicate_cols_count = gdf.columns.duplicated().sum()
    duplicate_rows_count = gdf.duplicated().sum()

    gdf.columns = [col.strip().lower() for col in gdf.columns]
    gdf = gdf.loc[:, ~gdf.columns.duplicated()]
    gdf = gdf.drop_duplicates()

    locals()[var_name] = gdf

    if duplicate_cols_count > 0 or duplicate_rows_count > 0:
        attribute_cleanup_report.append({
            "table": var_name,
            "duplicate_columns_removed": duplicate_cols_count,
            "duplicate_rows_removed": duplicate_rows_count
        })

if attribute_cleanup_report:
    print("Очистка атрибутивных данных выполнена:")
    for entry in attribute_cleanup_report:
        print(f"- {entry['table']}: удалено дублирующих колонок — {entry['duplicate_columns_removed']}, "
              f"дублирующих строк — {entry['duplicate_rows_removed']}")
else:
    print("Во всех таблицах атрибутивные данные уже были чистыми.")


Во всех таблицах атрибутивные данные уже были чистыми.


**Проверка на дубликаты**

для удобства отслеживания дубликатов сделаем 1 общий df

In [12]:
merged_list = []

for var_name in geo_vars:
    gdf = locals()[var_name]
    gdf = gdf.copy()
    gdf["source"] = var_name  # для отслеживания, откуда объект
    merged_list.append(gdf)

merged_gdf = pd.concat(merged_list, ignore_index=True)

merged_gdf = merged_gdf.drop_duplicates(subset=["geometry"])

теперь сама проверка, причем будем считать что объект-дубликат, если имя одинаковое и геометрия практически одинаковая(т.е разница достаточно мала, чтобы думать, что это один и тот же объект)

In [13]:
df = merged_gdf.copy()

geom_simple = []
for geom in df.geometry:
    if geom.geom_type == 'Point':
        geom_simple.append(Point(round(geom.x, 4), round(geom.y, 4)))
    else:
        geom_simple.append(geom.simplify(0.0001))  #для полигонов/линий

df["geom_simple"] = geom_simple

dups = df[df.duplicated(subset=["name", "geom_simple"], keep=False)]


if dups.empty:
    print(" Дубликаты не найдены.")
else:
    print(f"Найдено дубликатов: {len(dups)}")
    display(dups)

Найдено дубликатов: 2


,region,id_full,level,type,name,id,geometry,source,gicity,okrug,...,x,y,cpi_id,category,category_name,dates,free_speed_to_limit,length,geom_length,geom_simple
10486,NaN,NaN,NaN,NaN,"Уютный дом, арт-пространство",131677,POINT (4511028.461 9486267.147),education,NaN,NaN,...,40.523258,64.532398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4511028.4614 9486267.1472)
11882,NaN,NaN,NaN,NaN,"Уютный дом, арт-пространство",6801,POINT (4511028.461 9486267.147),tourism,NaN,NaN,...,40.523258,64.532398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4511028.4614 9486267.1472)


видим, что объект действительно один и тот же, но с разным назначением, поэтому удалять не будем (значит, в целом все данные проверены)

#Шаг 2. Создание гексогональной сетки

Создадим функцию гексагона

In [14]:
def create_hex(center_x, center_y, size):
    angles = [30, 90, 150, 210, 270, 330]
    coords = []
    for angle in angles:
        rad = np.deg2rad(angle)
        x = center_x + size * np.cos(rad)
        y = center_y + size * np.sin(rad)
        coords.append((x, y))
    return Polygon(coords)

center_x, center_y - координаты центра шестиугольника

size - расстояние от центра до любой вершины (радиус описанной окружности)

сгенерируем покрытие

In [15]:
def generate_hexs(gdf, hex_size):
    xmin, ymin, xmax, ymax = gdf.total_bounds

    hex_width = hex_size * np.sqrt(3)
    hex_height = 2 * hex_size
    row_spacing = hex_size * 1.5

    hexagons = []
    row = 0

    y = ymin - hex_height

    while y <= ymax + hex_height:

        x_offset = (hex_width / 2) if (row % 2) else 0  # смещение для четных/нечетных рядов
        x = xmin - hex_width + x_offset

        while x <= xmax + hex_width:
            hexagon = create_hex(x, y, hex_size)
            hexagons.append(hexagon)
            x += hex_width
        y += row_spacing
        row += 1

    return gpd.GeoDataFrame(geometry=hexagons, crs=gdf.crs)

In [16]:
hex_size = 15000  # в метрах
hex_grid_full = generate_hexs(region, hex_size)
hex_grid_clipped = gpd.overlay(hex_grid_full, region, how="intersection") #обрезка сетки по границам региона

region_wgs = region.to_crs(4326)
hex_grid_wgs = hex_grid_clipped.to_crs(4326)

распределим людей по гексагону соответственно площади

In [17]:
def distribute_population(pop_gdf, hex_grid):
    hex_grid = hex_grid.copy()
    if "hex_id" not in hex_grid.columns:
        hex_grid["hex_id"] = hex_grid.index

    pop_metric = pop_gdf.to_crs(3857)
    hex_metric = hex_grid.to_crs(3857)

    results = []

    for _, pop_row in pop_metric.iterrows():
        total_area = pop_row.geometry.area
        if total_area == 0:  # чтобы не делить на 0
            continue

        pop_value = pop_row['people']

        possible_matches_index = list(hex_metric.sindex.intersection(pop_row.geometry.bounds))
        if not possible_matches_index:
            continue

        possible_matches = hex_metric.iloc[possible_matches_index]

        for _, hex_row in possible_matches.iterrows():
            intersection = pop_row.geometry.intersection(hex_row.geometry)
            if not intersection.is_empty:

                fraction = intersection.area / total_area #доля площади
                results.append({        #часть населения
                    'hex_id': hex_row['hex_id'],
                    'pop_contrib': pop_value * fraction
                })

    if results:
        pop_df = pd.DataFrame(results)
        pop_sum = pop_df.groupby('hex_id')['pop_contrib'].sum().reset_index(name='population')
        pop_sum['population'] = pop_sum['population'].round().astype(int)
        return pop_sum
    else:
        return pd.DataFrame(columns=['hex_id', 'population'])


In [18]:
def count_hex(points_gdf, hex_grid, count_label):

    if "hex_id" not in hex_grid.columns:
        hex_grid = hex_grid.copy()
        hex_grid["hex_id"] = hex_grid.index

    joined = gpd.sjoin(points_gdf, hex_grid[["hex_id", "geometry"]],
                      how="left", predicate="within")

    counts = joined.groupby("hex_id").size().reset_index(name=count_label)
    return counts

In [19]:
hex_grid_clipped["hex_id"] = hex_grid_clipped.index
hex_population = distribute_population(population, hex_grid_clipped)

In [20]:
hex_zdrav = count_hex(zdrav, hex_grid_clipped, count_label="count_zdrav")
hex_education = count_hex(education, hex_grid_clipped, count_label="count_education")
hex_tourism = count_hex(tourism, hex_grid_clipped, count_label="count_tourism")
hex_buildings = count_hex(buildings, hex_grid_clipped, count_label = "count_buildings")

In [21]:
hex_grid_clipped["hex_id"] = hex_grid_clipped.index
hex_grid_combined = hex_grid_clipped.merge(hex_population, on='hex_id', how='left')
hex_grid_combined = hex_grid_combined.merge(hex_education, on='hex_id', how='left')
hex_grid_combined = hex_grid_combined.merge(hex_tourism, on='hex_id', how='left')
hex_grid_combined = hex_grid_combined.merge(hex_zdrav, on='hex_id', how='left')
hex_grid_combined = hex_grid_combined.merge(hex_buildings, on='hex_id', how='left')

for col in ["population","count_education", "count_tourism", "count_zdrav", "count_buildings"]:
    hex_grid_combined[col] = hex_grid_combined[col].fillna(0).astype(int)


In [22]:
people_gdf = people.to_crs(hex_grid_combined.crs)
people_gdf.columns = people_gdf.columns.str.lower()

In [23]:
type_col = [c for c in people_gdf.columns if c.endswith("type") or "settle" in c][0]
name_col = [c for c in people_gdf.columns if "name" in c][0]

In [24]:
def classify(txt):
    t = (txt or "").lower()
    if any(w in t for w in ("город", "г.", "пгт")): return "city"
    if any(w in t for w in ("село", "дерев", "поселок", "посёлок", "д.")): return "village"
    return ""

In [25]:
people_gdf["cls"] = people_gdf[type_col].apply(classify)
people_gdf[name_col] = people_gdf[name_col].fillna("")

In [26]:
cities   = people_gdf[people_gdf["cls"]=="city"].copy()
villages = people_gdf[people_gdf["cls"]=="village"].copy()

In [27]:
all_cities = cities.copy()
all_villages = villages.copy()

all_cities = all_cities.to_crs("EPSG:4326")
all_villages = all_villages.to_crs("EPSG:4326")

In [28]:
hex_grid_combined.fillna(0, inplace=True)
hex_grid_combined

,region,id_full,level,type,name,id,geometry,hex_id,population,count_education,count_tourism,count_zdrav,count_buildings
0,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((4363574.626 8545927.688, 4363694.306...",0,0,0,0,0,0
1,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((4454077.338 8546676.963, 4453945.158...",1,0,0,0,0,0
2,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((4575259.918 8543711.128, 4574508.53 ...",2,0,0,0,0,0
3,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((4590042.426 8550176.443, 4593616.036...",3,0,0,0,0,0
4,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((4605278.703 8543973.112, 4605278.996...",4,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11361,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((6538599.585 17010176.443, 6525609.20...",11361,0,0,0,0,0
11362,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((6564580.347 17010176.443, 6551589.96...",11362,0,0,0,0,0
11363,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((6590561.109 17010176.443, 6577570.72...",11363,0,0,0,0,0
11364,Архангельская область,3,2,Область,Архангельская,144105,"POLYGON ((6603551.49 17002676.443, 6590561.109...",11364,0,0,0,0,0


#Воспользуемся ML для построения аналитики по гексагонам.

Найдем самые лучшие и худшие территории. Так как у нас нет данных для обучения воспользуемся кластеризацией. Разделим на классы территории и попробуем написать модель, которая будет по известным признакам анализировать данные. Воспользуемся RandomForest. Это умная модель, но она не сильно зависит от качества данных. Гиперпараметры подберем с помощью gridsearch.

In [29]:
data_ml = hex_grid_combined.copy()
features = ['population', 'count_education', 'count_tourism', 'count_zdrav', 'count_buildings']

for feature in features:
  data_ml[feature] = data_ml[feature].replace(0, 0.1)
  data_ml[f'log_{feature}'] = np.log1p(data_ml[feature])
log_features = [f'log_{feat}' for feat in features]
X = data_ml[log_features].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Здесь мы логарифмируем, приводим данные к одному виду. Используем scaler чтобы посчитать все как нужно. Далее создадим 3 кластера и определим, какой соответствует высокому уровню развития территории

In [30]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)
cluster_means = [X_scaled[clusters == i].mean(axis=0).mean() for i in range(3)]
high_dev_cluster = np.argmax(cluster_means)


Создаем метки, делим выборку на тестовую и тренировочую, делим данные. Далее подберем параметры и обучим модель

In [31]:
data_ml['dev_class'] = clusters
data_ml['is_high_dev'] = (data_ml['dev_class'] == high_dev_cluster).astype(int)
X = data_ml[log_features]
y = data_ml['is_high_dev']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2']
}
rf = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
    oob_score=True
)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1,
    scoring='f1_weighted'
)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              oob_score=True, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1_weighted', verbose=1)

Теперь обучим модель и посмотрим на наши результаты

In [33]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)
data_ml['high_dev_proba'] = best_model.predict_proba(X)[:, 1]
top_10 = data_ml.nlargest(10, 'high_dev_proba')
bottom_10 = data_ml.nsmallest(10, 'high_dev_proba')
data_ml['dev_category'] = 'medium'
data_ml.loc[top_10.index, 'dev_category'] = 'top'
data_ml.loc[bottom_10.index, 'dev_category'] = 'bottom'

Отлично! Мы нашли лучшие и худшие регионы. Визуализацию добавим на карту, и посмторим на то, насколько хороша наша модель

In [35]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
from sklearn.metrics import classification_report

# Сначала выполняем все расчеты и создаем переменные
print("\nТоп-10 гексагонов:")
top_10_sorted = top_10[['hex_id', 'high_dev_proba']].sort_values('high_dev_proba', ascending=False).reset_index(drop=True)
print(top_10_sorted)

print("\nХудшие 10 гексагонов:")
bottom_10_sorted = bottom_10[['hex_id', 'high_dev_proba']].sort_values('high_dev_proba').reset_index(drop=True)
print(bottom_10_sorted)

# Создаем feature_importances
feature_importances = pd.Series(best_model.feature_importances_, index=log_features)
feature_importances = feature_importances.sort_values(ascending=False)
print("\nВажность признаков:")
print(feature_importances)

print("\nОтчет о классификации на тестовых данных:")
class_report = classification_report(y_test, best_model.predict(X_test))
print(class_report)
print(f"OOB-оценка: {best_model.oob_score_:.4f}")

# Теперь создаем PDF с графиками
with PdfPages('результаты_анализа.pdf') as pdf:
    
    # Первый график - карта регионов
    fig1, ax1 = plt.subplots(1, 1, figsize=(15, 10))
    
    colors = {'bottom': 'red', 'medium': 'yellow', 'top': 'green'}
    cmap = LinearSegmentedColormap.from_list('dev_cmap', ['red', 'yellow', 'green'])
    
    data_ml.plot(column='high_dev_proba',
                 ax=ax1,
                 cmap=cmap,
                 legend=True,
                 legend_kwds={'label': "Вероятность развитости", 'shrink': 0.7},
                 alpha=0.7)
    
    top_10.plot(ax=ax1, color='green', edgecolor='black', linewidth=1.5, label='Топ-10')
    bottom_10.plot(ax=ax1, color='red', edgecolor='black', linewidth=1.5, label='Худшие 10')
    
    plt.title('Уровень развития регионов (Random Forest)', fontsize=16)
    plt.axis('off')
    plt.legend(title='Категории', loc='lower right')
    plt.tight_layout()
    
    # Сохраняем первый график в PDF
    pdf.savefig(fig1, bbox_inches='tight')
    plt.close(fig1)
    
    # Второй график - важность признаков
    fig2, ax2 = plt.subplots(figsize=(12, 8))
    feature_importances.plot(kind='bar', ax=ax2)
    plt.title('Важность признаков в модели Random Forest', fontsize=16)
    plt.ylabel('Важность', fontsize=12)
    plt.xlabel('Признаки', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Сохраняем второй график в PDF
    pdf.savefig(fig2, bbox_inches='tight')
    plt.close(fig2)
    
    # Третья страница - текстовые результаты
    fig3 = plt.figure(figsize=(10, 12))
    plt.axis('off')
    
    # Подготавливаем текст
    text_content = []
    
    text_content.append("ТОП-10 ГЕКСАГОНОВ:")
    text_content.append(top_10_sorted.to_string())
    text_content.append("\n" + "="*50 + "\n")
    
    text_content.append("ХУДШИЕ 10 ГЕКСАГОНОВ:")
    text_content.append(bottom_10_sorted.to_string())
    text_content.append("\n" + "="*50 + "\n")
    
    text_content.append("ВАЖНОСТЬ ПРИЗНАКОВ:")
    text_content.append(feature_importances.to_string())
    text_content.append("\n" + "="*50 + "\n")
    
    text_content.append("ОТЧЕТ О КЛАССИФИКАЦИИ:")
    text_content.append(class_report)
    text_content.append(f"\nOOB-оценка: {best_model.oob_score_:.4f}")
    
    # Объединяем весь текст
    full_text = "\n".join(text_content)
    
    # Добавляем текст на страницу
    plt.figtext(0.05, 0.95, full_text, fontfamily='monospace', 
                fontsize=8, verticalalignment='top')
    plt.title('Результаты анализа', fontsize=16, pad=20)
    plt.tight_layout()
    
    # Сохраняем текстовую страницу
    pdf.savefig(fig3, bbox_inches='tight')
    plt.close(fig3)
    
    # Добавляем метаданные
    pdf_info = pdf.infodict()
    pdf_info['Title'] = 'Анализ развития регионов'
    pdf_info['Author'] = 'Аналитик'
    pdf_info['Subject'] = 'Результаты Random Forest модели'
    pdf_info['Keywords'] = 'регионы, machine learning, анализ'

print("Все графики и результаты сохранены в файл 'результаты_анализа.pdf'")


Топ-10 гексагонов:
   hex_id  high_dev_proba
0    2013        0.999934
1    1951        0.999922
2    1892        0.999891
3    2012        0.979934
4    1950        0.969934
5    2009        0.969796
6    1947        0.959796
7    1893        0.949799
8    2076        0.809828
9    1949        0.809801

Худшие 10 гексагонов:
   hex_id  high_dev_proba
0       0             0.0
1       1             0.0
2       2             0.0
3       3             0.0
4       4             0.0
5       5             0.0
6       6             0.0
7       7             0.0
8       8             0.0
9       9             0.0

Важность признаков:
log_count_education    0.290013
log_count_zdrav        0.290009
log_population         0.270008
log_count_tourism      0.129968
log_count_buildings    0.020002
dtype: float64

Отчет о классификации на тестовых данных:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2271
           1       0.75      1.00    

C:\Users\smirn\AppData\Local\Temp\ipykernel_5336\482800702.py:48: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  plt.legend(title='Категории', loc='lower right')
C:\Users\smirn\AppData\Local\Temp\ipykernel_5336\482800702.py:48: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(title='Категории', loc='lower right')


Все графики и результаты сохранены в файл 'результаты_анализа.pdf'


In [37]:
A= data_ml.to_crs(4326)

In [38]:
region_union = region_wgs.geometry.union_all()

cities_in_region = all_cities[all_cities.geometry.within(region_union)]
villages_in_region = all_villages[all_villages.geometry.within(region_union)]

centroid = region_union.centroid

centroid_wgs = gpd.GeoSeries([centroid], crs=region_wgs.crs).to_crs(epsg=4326).iloc[0]

In [39]:
def choose_color(row):
    color_dict = {
        "bottom": "red",
        "medium": "yellow",
        "top": "green",
    }
    return color_dict.get(row.get("dev_category"))

m1 = folium.Map(location=[centroid_wgs.y, centroid_wgs.x], zoom_start=5, tiles="OpenStreetMap")

folium.GeoJson(
    region_wgs,
    style_function=lambda x: {
        'fillColor': 'white',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.05
    }
).add_to(m1)

def style_function(feature, fill_color):
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }

for _, row in A.iterrows():
    fill_color = choose_color(row)
    sf = partial(style_function, fill_color=fill_color)

    folium.GeoJson(
        row["geometry"],
        style_function=sf,
        highlight_function=lambda feature: {
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0.7
        }
    ).add_to(m1)

In [40]:
from IPython.display import HTML
import folium

# Создаем HTML с картой и легендой
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Карта регионов</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        .legend {{
            font-size: 14px;
            padding: 10px;
            max-width: 400px;
            margin: 10px 0;
            border: 1px solid #ccc;
            background: white;
        }}
        .map-container {{ margin-bottom: 20px; }}
    </style>
</head>
<body>
    <h1>Уровень развития регионов</h1>
    
    <div class="map-container">
        {m1._repr_html_()}
    </div>
    
    <div class="legend">
        <b>Легенда:</b><br>
        <span style="color: red;">&#9632;</span> bottom<br>
        <span style="color: yellow;">&#9632;</span> medium<br>
        <span style="color: green;">&#9632;</span> top<br>
    </div>
</body>
</html>
"""

# Сохраняем в файл
with open('карта_с_легендой.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("HTML файл сохранен: карта_с_легендой.html")

HTML файл сохранен: карта_с_легендой.html


##Шаг 3

#Итоговая визуализация гексагонов

In [41]:
hex_grid_combined["tooltip_text"] = (
    "Население: " + hex_grid_combined["population"].astype(str) + "<br>" +
    "Образование: " + hex_grid_combined["count_education"].astype(str) + "<br>" +
    "Туризм: " + hex_grid_combined["count_tourism"].astype(str) + "<br>" +
    "Здравоохранение: " + hex_grid_combined["count_zdrav"].astype(str) + "<br>" +
    "Жилые дома: " + hex_grid_combined["count_buildings"].astype(str)
)

hex_grid_wgs = hex_grid_combined.to_crs(4326)

Выделим разными цветами распределение мест в гексагонах

In [42]:
def categorize_hex(row):
    educations = row["count_education"] > 0
    tourists = row["count_tourism"] > 0
    clinics = row["count_zdrav"] > 0
    buildings = row["count_buildings"] > 0

    if not (educations or tourists or clinics or buildings):
        return 'lightblue'       # нет объектов
    if educations and not tourists and not clinics and not buildings:
        return 'green'           # только объекты образования
    if tourists and not educations and not clinics and not buildings:
        return 'orange'          # только объекты туризма
    if clinics and not educations and not tourists and not buildings:
        return 'purple'          # только медучреждения
    if buildings and not educations and not tourists and not clinics:
        return 'darkgreen'          # только жилые постройки

    if buildings and educations and not tourists and not clinics:
        return 'fuchsia'          # жилые постройки и объекты образования
    if buildings and tourists and not clinics and not educations:
        return 'yellow'          # объекты туризма и жилые дома
    if buildings and clinics and not tourists and not educations:
        return '	olive'          # медучреждения и жилые дома
    if educations and tourists and not clinics and not buildings:
        return 'limegreen'          # объекты образования и туризма
    if educations and clinics and not tourists and not buildings:
        return 'turquoise'       # объекты образования и медучреждения
    if tourists and clinics and not educations and not buildings:
        return 'red'             # объекты туризма и медучреждения

    if educations and tourists and clinics and not buildings:
        return 'mediumpurple'         # образование, туризм и медицина
    if educations and tourists and buildings and not clinics:
        return 'gold'            # образование, туризм и жилые
    if educations and clinics and buildings and not tourists:
        return 'mediumturquoise' # образование, медицина и жилые
    if tourists and clinics and buildings and not educations:
        return 'darkorange'      # туризм, медицина и жилые
    if educations and tourists and clinics and buildings:
        return 'darkred'         # все четыре


In [43]:
region_union = region_wgs.geometry.union_all()

cities_in_region = all_cities[all_cities.geometry.within(region_union)]
villages_in_region = all_villages[all_villages.geometry.within(region_union)]

centroid = region_union.centroid

centroid_wgs = gpd.GeoSeries([centroid], crs=region_wgs.crs).to_crs(epsg=4326).iloc[0]

In [44]:
m = folium.Map(location=[centroid_wgs.y, centroid_wgs.x], zoom_start=5, tiles="OpenStreetMap")

folium.GeoJson(region_wgs, style_function=lambda x: {
    'fillColor': 'white',
    'color': 'black',
    'weight': 1,
    'fillOpacity': 0.05
}).add_to(m)

for _, row in hex_grid_wgs.iterrows():
    fill_color = categorize_hex(row)
    folium.GeoJson(row["geometry"], style_function=lambda x, fc=fill_color: {
            'fillColor': fc,
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7
        },

        highlight_function=lambda feature: {
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0.7
        },

        tooltip=folium.Tooltip(row["tooltip_text"], sticky=True)
    ).add_to(m)

for _, row in cities_in_region.iterrows():
    point = row.geometry.representative_point()
    folium.CircleMarker(
        location=[point.y, point.x],
        radius=4,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.9,
        popup=row[name_col]
    ).add_to(m)
for _, row in villages_in_region.iterrows():
    point = row.geometry.representative_point()
    folium.CircleMarker(
        location=[point.y, point.x],
        radius=2.8,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.8,
        popup=row[name_col]
    ).add_to(m)



In [45]:
import folium
from IPython.display import HTML
import os

# Создаем полный HTML файл с картой и легендой
full_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Карта объектов инфраструктуры</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f5f5f5;
        }}
        .container {{
            display: flex;
            gap: 20px;
            flex-wrap: wrap;
        }}
        .map-container {{
            flex: 1;
            min-width: 600px;
            height: 800px;
            border: 1px solid #ccc;
            background: white;
        }}
        .legend-container {{
            flex: 0 0 400px;
            background: white;
            padding: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
            height: fit-content;
        }}
        h1 {{
            color: #333;
            margin-bottom: 20px;
        }}
        .legend-item {{
            margin: 5px 0;
            line-height: 1.4;
        }}
        .section-title {{
            font-weight: bold;
            margin-top: 15px;
            margin-bottom: 5px;
            color: #333;
            border-bottom: 1px solid #eee;
            padding-bottom: 3px;
        }}
    </style>
</head>
<body>
    <h1>Карта объектов инфраструктуры</h1>
    
    <div class="container">
        <div class="map-container">
            {m._repr_html_()}
        </div>
        
        <div class="legend-container">
            <div class="section-title">Типы объектов:</div>
            <div class="legend-item"><span style="color: lightblue;">■</span> Нет объектов</div>
            <div class="legend-item"><span style="color: green;">■</span> Только образовательные учреждения</div>
            <div class="legend-item"><span style="color: orange;">■</span> Только объекты туризма</div>
            <div class="legend-item"><span style="color: purple;">■</span> Только медучреждения</div>
            <div class="legend-item"><span style="color: darkgreen;">■</span> Только жилые постройки</div>
            <div class="legend-item"><span style="color: fuchsia;">■</span> Жилые постройки и объекты образование</div>
            <div class="legend-item"><span style="color: yellow;">■</span> Объекты туризма и жилые дома</div>
            <div class="legend-item"><span style="color: olive;">■</span> Медучреждения и жилые дома</div>
            <div class="legend-item"><span style="color: limegreen;">■</span> Объекты образования и туризма</div>
            <div class="legend-item"><span style="color: turquoise;">■</span> Объекты образования и медицины</div>
            <div class="legend-item"><span style="color: red;">■</span> Объекты туризма и медицины</div>
            <div class="legend-item"><span style="color: mediumturquoise;">■</span> Объекты образование, медицины и жилые дома</div>
            <div class="legend-item"><span style="color: darkorange;">■</span> Объекты туризма, медицины и жилые дома</div>
            <div class="legend-item"><span style="color: gold;">■</span> Объекты образования, туризма и жилые дома</div>
            <div class="legend-item"><span style="color: darkred;">■</span> Все четыре</div>
            
            <div class="section-title">Населённые пункты:</div>
            <div class="legend-item"><span style="color: red;">●</span> Город</div>
            <div class="legend-item"><span style="color: black;">●</span> Село / деревня</div>
        </div>
    </div>
</body>
</html>
"""

# Сохраняем в файл
with open('карта_инфраструктуры_с_легендой.html', 'w', encoding='utf-8') as f:
    f.write(full_html)

print("Интерактивная карта сохранена: карта_инфраструктуры_с_легендой.html")

Интерактивная карта сохранена: карта_инфраструктуры_с_легендой.html
